# Auto Loan Prediction Dataset - Delicia Jathanna

## CSCI 3287 

In [ ]:
import sqlite3
import zipfile
import pandas as pd

## Connecting to my database & Creating the tables

In [ ]:
df_auto = pd.read_csv('auto_dataset.zip')

In [ ]:
conn = sqlite3.connect("autoloansdb.db")
cur = conn.cursor()

In [ ]:
drop_table_jobs = conn.execute('DROP TABLE IF EXISTS jobs;')

In [ ]:
create_jobs_table = conn.execute('''
CREATE TABLE IF NOT EXISTS jobs
	(
	  occupation VARCHAR(50) NOT NULL, --Occupation of the application
      salary INT NOT NULL, --Monthly income of the applicant
      credit_score INTEGER NOT NULL --Credit Score of the applicant
	);
''')

In [ ]:
drop_table_loan = conn.execute('DROP TABLE IF EXISTS loan;')

In [ ]:
create_loan_table = conn.execute('''
CREATE TABLE IF NOT EXISTS loan
	(
	  loan_id VARCHAR(50) PRIMARY KEY, --ID of the loan (primary key)
      occupation VARCHAR(50), --Occupation of the applicant (foreign key)
      FOREIGN KEY (occupation) REFERENCES jobs (occupation)
	);
''')

In [ ]:
drop_table_loan_status = conn.execute('DROP TABLE IF EXISTS loan_status;')

In [ ]:
create_loan_status_table = conn.execute('''
CREATE TABLE IF NOT EXISTS loan_status
	(
	  loan_id VARCHAR(50), --ID of the loan (foreign key)
      loan_status VARCHAR(50) CHECK (loan_status = 'Y' OR loan_status = 'N'), --Status of the loan
      credit_score INTEGER, --Credit Score of the applicant (foreign key)
      FOREIGN KEY (loan_id) REFERENCES loan (loan_id)
      FOREIGN KEY (credit_score) REFERENCES jobs (credit_score)
	);
''')

In [ ]:
drop_table_loan_term = conn.execute('DROP TABLE IF EXISTS loan_term;')

In [ ]:
create_loan_term_table = conn.execute('''
CREATE TABLE IF NOT EXISTS loan_term
	(
	  loan_id VARCHAR(50), --ID of the loan (foreign key)
      loan_term INTEGER, --Term of the loan in months 
      FOREIGN KEY (loan_id) REFERENCES loan_status (loan_id) ON INSERT CASCADE
	);
''')

In [ ]:
### Filling the tables with the data from the dataset

In [ ]:
for row in df_auto.to_dict('records'):
    cmd = ('''INSERT INTO jobs(occupation, salary, credit_score)\
    VALUES ( :Occupation, :Monthly_Income, :Credit_Score) ''' )
    conn.execute(cmd, row)

In [ ]:
for row in df_auto.to_dict('records'):
    cmd = ('''INSERT INTO loan(loan_id, occupation)\
    VALUES ( :Loan_ID, :Occupation) ''' )
    conn.execute(cmd, row)

In [ ]:
for row in df_auto.to_dict('records'):
    cmd = ('''INSERT INTO loan_status(loan_id, loan_status, credit_score)\
    VALUES ( :Loan_ID, :Loan_Status, :Credit_Score) ''' )
    conn.execute(cmd, row)

In [ ]:
for row in df_auto.to_dict('records'):
    cmd = ('''INSERT INTO loan_term(loan_id, loan_term)\
    VALUES ( :Loan_ID, :Loan_Amount_Term) ''' )
    conn.execute(cmd, row)

### Display the initial data in the tables

In [ ]:
res = cur.execute("SELECT * FROM jobs LIMIT 10")
res.fetchall()

In [ ]:
res = cur.execute("SELECT * FROM loan LIMIT 10")
res.fetchall()

In [ ]:
res = cur.execute("SELECT * FROM loan_status LIMIT 10")
res.fetchall()

In [ ]:
res = cur.execute("SELECT * FROM loan_term LIMIT 10")
res.fetchall()

# Index on loan_id

In [ ]:
drop_loan_id_index = conn.execute("DROP INDEX IF EXISTS loan_id_index")

In [ ]:
loan_id_index = conn.execute("CREATE INDEX loan_id_index ON loan(loan_id)")

# Queries

Now that we've added some indexes and triggers to ensure the dataset runs smoothly and meets the requirements, lets run some queries on them.

This first query will join all the ables together. This will take a subquery or using CTEs because not all of the tables have the same relationships. 

In [ ]:
all_table_join = cur.execute("""
    WITH 
    CTE1 AS (
            SELECT loan_id AS Loan_ID, jobs.occupation AS occupation, jobs.salary AS Salary, jobs.credit_score AS [Credit Score]
            FROM loan 
                JOIN jobs on jobs.occupation = loan.occupation
            ),
    CTE2 AS (
            SELECT loan.loan_id, loan_status AS Status
            FROM loan_status
                JOIN loan on loan.loan_id = loan_status.loan_id
            ),
    CTE3 AS (
            SELECT loan.loan_id, loan_term AS Term
            FROM loan_term 
                JOIN loan on loan.loan_id = loan_term.loan_id
            ) 
    SELECT CTE1.loan_id, Occupation, Salary, [Credit Score], CTE2.Status, CTE3.Term
    FROM  CTE1
          JOIN CTE2 ON CTE1.loan_id = CTE2.loan_id
          JOIN CTE3 ON CTE1.loan_id = CTE3.loan_id
    GROUP BY Occupation
    ORDER BY Occupation
    LIMIT 10;
    """)

all_table_join.fetchall()

##### Now lets run some requries that show what salary, credit score, and occupation are more favorable to be approved for an auto loan.

In [ ]:
favorable_options = cur.execute("""
    WITH 
    CTE1 AS (
            SELECT loan_id AS Loan_ID, jobs.occupation AS occupation, jobs.salary AS Salary, jobs.credit_score AS [Credit Score]
            FROM loan 
                JOIN jobs on jobs.occupation = loan.occupation
            ),
    CTE2 AS (
            SELECT loan.loan_id, loan_status AS Status
            FROM loan_status
                JOIN loan on loan.loan_id = loan_status.loan_id
            )
    SELECT CTE1.loan_id, Occupation, Salary, [Credit Score], CTE2.Status 
    FROM  CTE1
        JOIN CTE2 on CTE1.loan_id = CTE2.loan_id
    WHERE Status = 'Y'
    GROUP BY Occupation
    ORDER BY [Credit Score] desc
    LIMIT 10;
    """)

favorable_options.fetchall()

##### Now lets do the opposite

In [ ]:
unfavorable_options = cur.execute("""
    WITH 
    CTE1 AS (
            SELECT loan_id AS Loan_ID, jobs.occupation AS occupation, jobs.salary AS Salary, jobs.credit_score AS [Credit Score]
            FROM loan 
                JOIN jobs on jobs.occupation = loan.occupation
            ),
    CTE2 AS (
            SELECT loan.loan_id, loan_status AS Status
            FROM loan_status
                JOIN loan on loan.loan_id = loan_status.loan_id
            )
    SELECT CTE1.loan_id, Occupation, Salary, [Credit Score], CTE2.Status AS Status
    FROM  CTE1
    JOIN CTE2 on CTE1.loan_id = CTE2.loan_id
    WHERE Status = 'N'
    GROUP BY Occupation
    ORDER BY [Credit Score] 
    LIMIT 10;
    """)

unfavorable_options.fetchall()

# Triggers and Indexes
We will now Add some triggers and indexes to the data so that the queries run smoother and we can ensure that when a foreign key gets deleted, it gets deleted in every table.

In [ ]:
drop_fk_occupation_trigger = conn.execute("DROP TRIGGER IF EXISTS fk_occupation_delete")

In [ ]:
fk_occupation_trigger = conn.execute('''
    CREATE TRIGGER fk_occupation_delete
    AFTER DELETE ON jobs 
    FOR EACH ROW
    BEGIN
        DELETE FROM loan
        WHERE occupation = old.occupation;
    END;
    ''')

In [ ]:
drop_fk_loanid_trigger = conn.execute("DROP TRIGGER IF EXISTS fk_loanid_delete")

In [ ]:
fk_loanid_trigger = conn.execute('''
    CREATE TRIGGER fk_loanid_delete
    AFTER DELETE ON loan 
    FOR EACH ROW
    BEGIN
        DELETE FROM loan_term
        WHERE loan_id = old.loan_id;
        DELETE FROM loan_status
        WHERE loan_id = old.loan_id;
    END;
    ''')

In [ ]:
drop_fk_score_trigger = conn.execute("DROP TRIGGER IF EXISTS fk_score_delete")

In [ ]:
fk_score_trigger = conn.execute('''
    CREATE TRIGGER fk_score_delete
    AFTER DELETE ON jobs 
    FOR EACH ROW
    BEGIN
        DELETE FROM loan
        WHERE credit_score = old.credit_score;
        DELETE FROM loan_status
        WHERE credit_score = old.credit_score;
    END;
    ''')

### Testing the occupation foreign key trigger

In [ ]:
res = cur.execute("DELETE FROM jobs WHERE occupation = 'Teacher'")

In [ ]:
res = cur.execute("SELECT * FROM jobs LIMIT 10")
res.fetchall()

In [ ]:
res = cur.execute("SELECT * FROM loan LIMIT 10")
res.fetchall()

### Testing the loan_id trigger

In [ ]:
res = cur.execute("DELETE FROM loan WHERE loan_id = 'LP001005'")

In [ ]:
res = cur.execute("SELECT * FROM loan LIMIT 10")
res.fetchall()

In [ ]:
res = cur.execute("SELECT * FROM loan_term LIMIT 10")
res.fetchall()

In [ ]:
res = cur.execute("SELECT * FROM loan_status LIMIT 10")
res.fetchall()

### Testing the credit_score trigger

In [ ]:
res = cur.execute("DELETE FROM jobs WHERE credit_score < 675 ")

In [ ]:
res = cur.execute("SELECT * FROM jobs LIMIT 10")
res.fetchall()

In [ ]:
res = cur.execute("SELECT * FROM loan LIMIT 10")
res.fetchall()

In [ ]:
res = cur.execute("SELECT * FROM loan_status LIMIT 10")
res.fetchall()

Now that we've ensured that when a foreign key gets deleted, it deletes in every table, lets add a trigger to the loan status table that a loan cannot be approved if the credit score is below 675. 

In [ ]:
drop_loan_status_trigger = conn.execute("DROP TRIGGER IF EXISTS loan_status_trigger")

In [ ]:
loan_status_trigger = conn.execute('''
    CREATE TRIGGER loan_status_trigger
    AFTER INSERT ON loan_status 
    FOR EACH ROW
    BEGIN
        SELECT CASE 
            WHEN ((SELECT loan_status FROM loan_status WHERE loan_status.loan_id = NEW.loan_id AND loan_status.credit_score < 675) = 'Y') 
            THEN RAISE(ABORT, 'Credit Score must be above 675 to be approved for a loan') 
        END;
    END;
    ''')

In [ ]:
test_loan_status_trigger = conn.execute("INSERT INTO loan_status(loan_id, credit_score, loan_status) VALUES('LP002000', 650, 'Y')")

In [ ]:
cur.close()